https://ropiens.tistory.com/83

## 어노테이션 txt 파일 생성
- 원본 사진을 구글 드라이브에 업로드 후 txt파일 추출
- 너무 오래 걸림 (업로드 5시간)
- 원본 데이터를 yolov5에 클론 하는데 세월아 내월아 걸림
- 로컬 드라이브에서 진행하고 압축파일만 드라이브에 올리는거 추천

In [2]:
# Google Drive를 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 파일의 인코딩을 자동으로 감지하고 해당 인코딩을 사용하여 파일을 읽어 들이는 chardet 라이브러리를 사용
!pip install chardet

In [ ]:
import json
import chardet
import os
import zipfile

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

def json_to_yolo(json_file, yolo_dir, img_width, img_height):
    encoding = detect_encoding(json_file)
    with open(json_file, 'r', encoding=encoding) as f:
        data = json.load(f)

    # 각각의 txt 파일을 생성
    for item in data:
        img_file_name = item['images']['img_file_nm']
        txt_file_name = img_file_name.rsplit('.', 1)[0] + '.txt'  # 확장자를 .txt로 변경
        with open(f"{yolo_dir}/{txt_file_name}", 'w') as out_file:
            for ann in item['annotations']:
                lbl_id = int(ann['lbl_id'])  # 클래스 인덱스
                x_lower_left, y_lower_left, width, height = [
                    coord / (img_width if i % 2 == 0 else img_height)
                    for i, coord in enumerate(ann['annotations_info'])
                ]
                x_center = x_lower_left + (width / 2)
                y_center = y_lower_left + (height / 2)
                out_file.write(f"{lbl_id} {x_center} {y_center} {width} {height}\n")

    # 생성된 txt 파일들을 압축
    zip_filename = f"{yolo_dir}.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(yolo_dir):
            for file in files:
                if file.endswith('.txt'):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(yolo_dir, '..')))

    return zip_filename  # 압축 파일의 경로를 반환

In [ ]:
%mkdir /content/drive/MyDrive/ai_project/test/labels

In [ ]:
train_json_path = '/content/drive/MyDrive/ai_project/train/train.json'
test_json_path = '/content/drive/MyDrive/ai_project/test/test.json'
output_train_path = '/content/drive/MyDrive/ai_project/train/labels'
output_test_path = '/content/drive/MyDrive/ai_project/test/labels'

# train.json 파일 처리 및 압축
#train_zip_path = json_to_yolo(train_json_path, output_train_path, img_width=1920, img_height=1080)
#print(f"Train labels zip file created at: {train_zip_path}")

# test.json 파일 처리 및 압축
test_zip_path = json_to_yolo(test_json_path, output_test_path, img_width=1920, img_height=1080)
print(f"Test labels zip file created at: {test_zip_path}")


Test labels zip file created at: /content/drive/MyDrive/ai_project/test/labels.zip


In [ ]:
train_images_path = "/content/drive/MyDrive/ai_project/train/train_images"
train_labels_path = "/content/drive/MyDrive/ai_project/train/train_labels"

test_images_path = "/content/drive/MyDrive/ai_project/test/test_images"
test_labels_path = "/content/drive/MyDrive/ai_project/test/test_labels"

In [ ]:
import os

# Train 데이터
train_images = [os.path.join(train_images_path, f) for f in os.listdir(train_images_path) if f.endswith('.jpg') or f.endswith('.png')]
train_annotations = [os.path.join(train_labels_path, f) for f in os.listdir(train_labels_path) if f.endswith('.txt')]

# Test 데이터
test_images = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.endswith('.jpg') or f.endswith('.png')]
test_annotations = [os.path.join(test_labels_path, f) for f in os.listdir(test_labels_path) if f.endswith('.txt')]

# 라벨링 숫자 변경 (0-5)

In [9]:
!unzip "/content/drive/MyDrive/ai_project/train/train_labels/center_labels_train.zip" -d "/content/train"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: /content/train/CIT-SDB-3044_0003859.txt  
 extracting: /content/train/CIT-SDB-3044_0003860.txt  
 extracting: /content/train/CIT-SDB-3044_0003862.txt  
 extracting: /content/train/CIT-SDB-3044_0003863.txt  
 extracting: /content/train/CIT-SDB-3044_0003864.txt  
 extracting: /content/train/CIT-SDB-3044_0003865.txt  
 extracting: /content/train/CIT-SDB-3044_0003866.txt  
 extracting: /content/train/CIT-SDB-3044_0003867.txt  
 extracting: /content/train/CIT-SDB-3044_0003868.txt  
 extracting: /content/train/CIT-SDB-3044_0003869.txt  
 extracting: /content/train/CIT-SDB-3044_0003870.txt  
 extracting: /content/train/CIT-SDB-3044_0003871.txt  
 extracting: /content/train/CIT-SDB-3044_0003873.txt  
 extracting: /content/train/CIT-SDB-3044_0003874.txt  
 extracting: /content/train/CIT-SDB-3044_0003875.txt  
 extracting: /content/train/CIT-SDB-3044_0003876.txt  
 extracting: /content/train/CIT-SDB-3044_0003877.txt  
 extracting: /content/train/C

In [10]:
from glob import glob
train_img_list = glob('/content/train/*.txt')

print(len(train_img_list))

14000


In [11]:
import os

# 클래스 ID 매핑
class_mapping = {
    '115': '0',  # Person(보행자)
    '116': '1',  # Bicycle(자전거)
    '117': '2',  # Motorcycle(오토바이)
    '118': '3',  # Truck(트럭)
    '121': '4',  # Car(승용, 승합)
    '122': '5',  # Bus(버스)

}

# 레이블 파일이 위치한 디렉토리
label_dir = '/content/train'

# 모든 .txt 파일을 순회하며 클래스 인덱스를 변경합니다.
for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):
        label_file_path = os.path.join(label_dir, label_file)
        with open(label_file_path, 'r') as file:
            lines = file.readlines()

        with open(label_file_path, 'w') as file:
            for line in lines:
                parts = line.strip().split()
                class_id = parts[0]
                if class_id in class_mapping:
                    parts[0] = class_mapping[class_id]  # 클래스 인덱스를 새 값으로 변경
                file.write(' '.join(parts) + '\n')


In [12]:
import os
import zipfile

# 레이블 파일이 위치한 디렉토리
label_dir = '/content/train'
# 압축 파일의 경로와 이름 설정
zip_filename = '/content/drive/MyDrive/ai_project/center_labels0to5_train.zip'

# 새로운 ZIP 파일 생성
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # 레이블 파일들을 ZIP 파일에 추가
    for foldername, subfolders, filenames in os.walk(label_dir):
        for filename in filenames:
            if filename.endswith('.txt'):
                # 파일 경로 생성
                filepath = os.path.join(foldername, filename)
                # ZIP 파일에 파일 추가
                zipf.write(filepath, os.path.relpath(filepath, label_dir))

print(f"Created ZIP file: {zip_filename}")


Created ZIP file: /content/drive/MyDrive/ai_project/center_labels0to5_train.zip


# 이미지 사이즈 변경


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
from sklearn.model_selection import KFold

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -U -r requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 16062, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 16062 (delta 1), reused 2 (delta 0), pack-reused 16056
Receiving objects: 100% (16062/16062), 14.68 MiB | 21.17 MiB/s, done.
Resolving deltas: 100% (11029/11029), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.6/283.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB

In [ ]:
# 압축 해제할 위치 생성 및 이동
%mkdir /content/yolov5/ai_project
%mkdir /content/yolov5/ai_project/train
%mkdir /content/yolov5/ai_project/valid
%mkdir /content/yolov5/ai_project/test

%mkdir /content/yolov5/ai_project/train/images
%mkdir /content/yolov5/ai_project/train/labels

%mkdir /content/yolov5/ai_project/valid/images
%mkdir /content/yolov5/ai_project/valid/labels
%cd /content/yolov5/ai_project

/content/yolov5/ai_project


In [ ]:
# 압축 해제
!unzip "/content/drive/MyDrive/ai_project/train/train_images/CIT-SDB-3043.zip" -d "/content/yolov5/ai_project/train/images"
!unzip "/content/drive/MyDrive/ai_project/train/train_images/CIT-SDB-3044.zip" -d "/content/yolov5/ai_project/train/images"
!unzip "/content/drive/MyDrive/ai_project/train/train_images/CIT-SDB-3045.zip" -d "/content/yolov5/ai_project/train/images"
!unzip "/content/drive/MyDrive/ai_project/train/labels/labels_for_yolo.zip" -d "/content/yolov5/ai_project/train/labels"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0004853.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0005260.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0005590.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3044_0004902.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3044_0008581.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0006766.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0002016.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0005311.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0008287.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3044_0008807.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3044_0004374.txt  
 extracting: /content/yolov5/ai_project/train/labels/CIT-SDB-3043_0008649.txt  
 ext

In [ ]:
from glob import glob
train_img_list = glob('/content/yolov5/ai_project/test/images/*.jpg')
test_img_list = glob('/content/yolov5/ai_project/train/images/*.jpg')
valid_img_list = glob('/content/yolov5/ai_project/valid/images/*.jpg')

print(len(train_img_list), len(test_img_list), len(valid_img_list))

0 14000 0


In [ ]:
import cv2
import os

def resize_images_with_opencv(input_folder, output_folder, max_size=416):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 이미지를 OpenCV로 읽기
            img = cv2.imread(input_path)

            # Resize the image to 640x640 without maintaining the aspect ratio
            resized_img = cv2.resize(img, (640, 640))

            # 비율 유지하면서 이미지 크기 조정
            # height, width = img.shape[:2]
            # scale = min(max_size / width, max_size / height)
            # new_width, new_height = int(width * scale), int(height * scale)
            # resized_img = cv2.resize(img, (new_width, new_height))

            # 조정된 이미지 저장
            cv2.imwrite(output_path, resized_img)

# 사용 예시
%mkdir /content/train
resize_images_with_opencv('/content/yolov5/ai_project/train/images', '/content/train')

mkdir: cannot create directory ‘/content/train’: File exists


In [ ]:
from glob import glob
train_list = glob('/content/train/*.jpg')
print(len(train_list))

14000


In [ ]:
!zip -r /content/train.zip /content/train

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  adding: content/train/CIT-SDB-3043_0002678.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3044_0007770.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3044_0005397.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3044_0004787.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3044_0001719.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3045_0000292.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3045_0000297.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3043_0005357.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3044_0007498.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3043_0001001.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3043_0007308.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3043_0002899.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3043_0002946.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3044_0006110.jpg (deflated 0%)
  adding: content/train/CIT-SDB-3043_0005511.jpg (deflated 0%)
  adding: content/t

In [ ]:
!mv /content/train.zip /content/drive/MyDrive/

### 파일안의 이미지만 복사하는 코드

In [ ]:
import zipfile
import os

def zip_jpg_files_in_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in os.listdir(folder_path):
            if file.lower().endswith('.jpg'):
                file_path = os.path.join(folder_path, file)
                zipf.write(file_path, file)

# 사용 예시

zip_jpg_files_in_folder('/content/yolov5/ai_project/train', 'your_folder.zip')


In [ ]:
import shutil

# 압축 파일 경로
zip_file = '/content/yolov5/ai_project/your_folder.zip'

# Google 드라이브 내의 원하는 경로
destination = '/content/drive/MyDrive'

# 파일 복사
shutil.copy(zip_file, destination)
